In [ ]:
#import libraries
import boto3
import json
import time

In [ ]:
# Connection to AWS
folder = '/home/propietari/Documents/claus/' # TODO: change to your local path
file_name = 'AWS_S3_keys_JordiPlanas_Made_in_game.json' # TODO: Change to your filename
file = folder + file_name

with open(file, 'r') as dict:
    credentials = json.load(dict)
                                      
KEY = list(credentials)[0]
SECRET = list(credentials.values())[0]
s3BucketName = "wri-testing"
region = 'eu-central-1'

s3 = boto3.resource(
    service_name = 's3',
    region_name = region,
    aws_access_key_id = KEY,
    aws_secret_access_key = SECRET
)

In [ ]:
def startJob(s3BucketName, objectName):
    response = None
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.start_document_text_detection(DocumentLocation={
        'S3Object': {'Bucket': s3BucketName,
                     'Name': objectName
                    }
        })
    return response["JobId"]
def isJobComplete(jobId):
 # For production use cases, use SNS based notification 
 # Details at: https://docs.aws.amazon.com/textract/latest/dg/api-async.html
    time.sleep(5)
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    status = response["JobStatus"]
    print("Job status: {}".format(status))
    while(status == "IN_PROGRESS"):
        time.sleep(5)
        response = client.get_document_text_detection(JobId=jobId)
        status = response["JobStatus"]
        print("Job status: {}".format(status))
    return status
def getJobResults(jobId):
    pages = []
    client = boto3.client('textract',
#                           service_name = 's3',
                          region_name = region,
                          aws_access_key_id = KEY,
                          aws_secret_access_key = SECRET)
    response = client.get_document_text_detection(JobId=jobId)
    
    pages.append(response)
    print("Resultset page recieved: {}".format(len(pages)))
    nextToken = None
    if('NextToken' in response):
        nextToken = response['NextToken']
        while(nextToken):
            response = client.get_document_text_detection(JobId=jobId, NextToken=nextToken)
            pages.append(response)
            print("Resultset page recieved: {}".format(len(pages)))
            nextToken = None
            if('NextToken' in response):
                nextToken = response['NextToken']
    return pages


In [ ]:
def AWS_pdf_to_text(key, s3BucketName):
    # Document
    documentName = key
    jobId = startJob(s3BucketName, documentName)
    print("Started job with id: {}".format(jobId))
    if(isJobComplete(jobId)):
        response = getJobResults(jobId)
    #print(response)
    # Print detected text
    policy = ""
    for resultPage in response:
        for item in resultPage["Blocks"]:
            if item["BlockType"] == "LINE":
                policy = policy + item["Text"] + "\n"
                print ('\033[94m' + item["Text"] + '\033[0m')
    return policy

In [ ]:
i = 0
lower_limit = 0
upper_limit = lower_limit + 2
for obj in s3.Bucket(s3BucketName).objects.all():#.filter(Prefix='El Salvador'):
    if "India_image.pdf" in obj.key: #lower_limit < i < upper_limit:
        print(i, "**", obj.key)
        policy = AWS_pdf_to_text(obj.key, s3BucketName)
    i += 1

In [ ]:
policy